In [ ]:
#!pip install ../input/nnunet-wheels/MedPy-0.4.0-py3-none-any.whl  

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm.notebook import tqdm
from pydicom.data import get_testdata_file
from PIL import Image

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.utils import shuffle
import pydicom
from pydicom.data import get_testdata_file
import cv2
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import copy
from torchvision import models, transforms
from torch.optim import lr_scheduler

import os
import numpy as np
import pandas as pd
from medpy.io import load
from skimage.transform import resize



test_path = "/kaggle/input/rsna-breast-cancer-detection/test.csv"
test_img_dir = "/kaggle/input/rsna-breast-cancer-detection/test_images"
test_df = pd.read_csv(test_path)
test = []
target_size = (128, 128, 1)


for i in range(len(test_df)):
    line = test_df.iloc[i]
    p_id = str(line["patient_id"])
    im_id = str(line["image_id"]) + '.dcm'
    #prediction_id.append(line["prediction_id"])
    path = os.path.join(test_img_dir, p_id, im_id)
        #im = pydicom.dcmread(path).pixel_array
        #im = np.array(im)
        #im = cv2.resize(im, (128, 128))
        #im = im.reshape((128,128,1))
    data, header = load(path)
    resized_data = resize(data, target_size, anti_aliasing=True)
    test.append(resized_data)
    
test = np.array(test)

In [ ]:
output_dir ="/kaggle/temp/hiddentests"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import SimpleITK as sitk
import numpy as np
from skimage import filters
import matplotlib.pyplot as plt

subtest_path = "/kaggle/input/rsna-breast-cancer-detection/test.csv"
subtest_img_dir = "/kaggle/input/rsna-breast-cancer-detection/test_images"
global start_time
start_time= time.time()
test_df = pd.read_csv(subtest_path)
prediction_id = []
hiddin_test = []
cancer = []
for i in range(len(test_df)):
    line = test_df.iloc[i]
    p_id = str(line["patient_id"])
    im_id = str(line["image_id"]) + '.dcm'
    #prediction_id.append(line["prediction_id"])
    path = os.path.join(subtest_img_dir, p_id, im_id)

    # Load DICOM image using SimpleITK
    image = sitk.ReadImage(path)

    # Get the original image size
    original_size = image.GetSize()

    # Calculate the size of the resized image
    new_size = [128, 128, original_size[2]]

    # Resize image
    resized_image = sitk.Resample(image, original_size, sitk.Transform(), sitk.sitkLinear, image.GetOrigin(), image.GetSpacing(), image.GetDirection(), 0.0, image.GetPixelIDValue())

    # Convert resized image from SimpleITK object to numpy array
    np_image = sitk.GetArrayFromImage(resized_image)

    # Normalize image intensity values
    np_image = (np_image - np.min(np_image)) / (np.max(np_image) - np.min(np_image))

    # Remove noise from image
    np_image = filters.median(np_image)

    # Convert numpy array to image
    img = Image.fromarray((np_image[np_image.shape[0]//2] * 255).astype(np.uint8))
    
    output_path = os.path.join(output_dir, im_id[:-4] +  ".png")
    #im.save(output_path)
    # Save image as PNG
    img.save(output_path)
    

In [ ]:
def get_image_label(img_path):
    X = []
    for file in tqdm(os.listdir(img_path)):
        if file.endswith('.png'):
            im = cv2.imread(os.path.join(img_path, file), 0)
            im = cv2.resize(im, (128, 128))  # reduce to 128 x 128 
            im = im.reshape((128, 128, 1))
            #im = im / 225.0
            X.append(im)
            del im
    return X



            
img_path = "/kaggle/temp/hiddentests"
X = get_image_label(img_path)
final_test = np.array(X)

In [ ]:
final_test.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load the saved checkpoint
model = tf.keras.models.load_model("/kaggle/input/muslim/model_checkpoint.h5")

In [ ]:
test_path = "/kaggle/input/rsna-breast-cancer-detection/test.csv"
test_img_dir = "/kaggle/input/rsna-breast-cancer-detection/test_images"
test_df = pd.read_csv(test_path)
prediction_id = []
for i in range(len(test_df)):
    line = test_df.iloc[i]
    prediction_id.append(line["prediction_id"])
        

In [2]:
def read_xray(file_path, img_size=None):
    dicom = pydicom.read_file(file_path)
    img = dicom.pixel_array

    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = np.max(img) - img

    if img_size:
        img_size = (img_size, img_size) # Convert img_size to a tuple
        img = cv2.resize(img, dsize=img_size)

    # Add channel dim at First
    img = img[np.newaxis]

    # Converting img to float32
    
    img = img / np.max(img)
    img = img.astype("float32")

    return img

In [9]:
#Convert Test Images to Numpy arrays

test_path = "/kaggle/input/rsna-breast-cancer-detection/test_images/"
all_paths = []

test_df = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/test.csv")

for k in range(len(test_df)):
    row = test_df.iloc[k, :]
    all_paths.append(test_path + str(row.patient_id) + "/" + str(row.image_id) + ".dcm")
    
test_df["path"] = all_paths
print(test_df.head)

<bound method NDFrame.head of    site_id  patient_id    image_id laterality view  age  implant  machine_id  \
0        2       10008   736471439          L  MLO   81        0          21   
1        2       10008  1591370361          L   CC   81        0          21   
2        2       10008    68070693          R  MLO   81        0          21   
3        2       10008   361203119          R   CC   81        0          21   

  prediction_id                                               path  
0       10008_L  /kaggle/input/rsna-breast-cancer-detection/tes...  
1       10008_L  /kaggle/input/rsna-breast-cancer-detection/tes...  
2       10008_R  /kaggle/input/rsna-breast-cancer-detection/tes...  
3       10008_R  /kaggle/input/rsna-breast-cancer-detection/tes...  >


In [10]:
X_test = []
for i in range(test_df.shape[0]):
    row = test_df.iloc[i, :]
    path = test_path + str(row.patient_id) + "/" + str(row.image_id) + ".dcm"
    xray = read_xray(path, 128)
    X_test.append(xray)
    
print(len(X_test))

4


In [8]:
print(len(X_test))

12


In [13]:
test_csv = pd.read_csv('../input/rsna-breast-cancer-detection/test.csv')
submission = pd.DataFrame(data={'prediction_id': test_csv['prediction_id'],
                                'cancer': np.random.rand(test_csv.shape[0])}).drop_duplicates(subset='prediction_id')
submission.head()
submission.to_csv('submission.csv', index=False)

In [14]:
submission

,prediction_id,cancer
0,10008_L,0.220714
2,10008_R,0.056728


In [ ]:
#import time

#start_time = time.time()
#y = 
#end_time = time.time()
#elapsed1_time = end_time - start_time

In [ ]:
#print(elapsed1_time)

In [ ]:
import time
max_time = 5*60*60 # max runtime

for i in range(len(test_df)):
    duration = time.time()-start_time
    if duration <  max_time:
        #start_time = time.time()
        prob = model.predict(final_test)[i][0]
        cancer.append(prob)
    else:
        submission = pd.DataFrame(data={"prediction_id": prediction_id, "cancer": cancer})
        submission = submission.sort_values("cancer", ascending=False)
        submission = submission.drop_duplicates(subset="prediction_id", keep="first")
        submission.to_csv("submission.csv", index=False)
#end_time = time.time()
#elapsed_time = end_time - start_time

In [ ]:
#print(elapsed_time)

submission = pd.DataFrame(data={"prediction_id": prediction_id, "cancer": cancer})
submission = submission.sort_values("cancer", ascending=False)

submission = submission.drop_duplicates(subset="prediction_id", keep="first")
submission.to_csv("submission.csv", index=False)
submission

submission = pd.DataFrame(data={"prediction_id": prediction_id, "cancer": cancer})
submission = submission.sort_values("cancer", ascending=False)
submission = submission.drop_duplicates(subset="prediction_id", keep="first")
submission.to_csv("submission.csv", index=False)